# Tarea 3: Derivación automática 1

Fecha de envío del PR inicial: **lunes 27 de marzo, antes de la clase**

Fecha de aceptación del PR: **martes 4 de abril, antes de la clase**

## Ejercicio 1

Implementen una nueva estructura paramétrica (`type`) que llamaremos `Dual` y que defina los duales, donde el parámetro debe ser un subtipo de `Real`; la siguiente celda sirve para empezar. La parte que identifica a $f_0$ será llamada `fun`, y la correspondiente a $f'_0$ será `der`.

La definición debe incluir métodos que sean compatibles con las dos propiedades arriba mencionadas, es decir, que el dual de una constante (cualquier número real) sea $(c,0)$, y que el de la variable independiente sea $(x_0,1)$. Para lo segundo definiremos una función `xdual` con la propiedad mencionada.

---

In [1]:
"""
    Dual{T<:Real}

Definición de los duales, donde lo campos son:
"""
type Dual{T<:Real}
    fun::T
    der::T
end

Dual

In [2]:
#= 
Definan un método que permita la promoción automática cuando 
las entradas para definir el dual no son del mismo tipo
=#
"""
promoviendo los duales del tipo distinto
"""
function Dual(fun,der)
 return Dual(promote(fun,der)...)
end

Dual

In [3]:
#= 
Aquí se define un método que garantiza que el dual de una constante 
(número) cumple lo requerido
=#
function Dual(a::Real)
    return Dual(a,0.0)
end

Dual{T<:Real}

In [4]:
#= 
Aquí se define la función `xdual`, que se usará para identificar 
la variable independiente. La función dependerá de x_0, y debe 
regresar el Dual apropiado a la variable independiente
=#

"""
    xdual(x0) -> Dual(x0, 1)

...
"""
function xdual(x0)
    return Dual(x0,1)
end

xdual

In [5]:
#= 
Muestren que su código funciona con tests adecuados para crear duales,
para promoverlos, y al definir el dual de un número y `xdual`.

En esto es útil usar la infraestructura de Julia; ver:
https://julia.readthedocs.io/en/stable/stdlib/test/
=#
using Base.Test
@testset "pruebas realizadas" begin
a = Dual(1, 2.0)
@test a.fun == 1.0
@test a.der == 2.0

using Base.Test
a=Dual(1,2.0)
@test a.fun==1
@test a.der==2

b=Dual(3)
@test b.fun==3
@test b.der==0


c=xdual(10)
@test c.fun==10
@test c.der==1
end

Test Summary:      | Pass  Total
  pruebas realizadas |    8      8


Base.Test.DefaultTestSet("pruebas realizadas",Any[Test Passed
  Expression: a.fun == 1.0
   Evaluated: 1.0 == 1.0,Test Passed
  Expression: a.der == 2.0
   Evaluated: 2.0 == 2.0,Test Passed
  Expression: a.fun == 1
   Evaluated: 1.0 == 1,Test Passed
  Expression: a.der == 2
   Evaluated: 2.0 == 2,Test Passed
  Expression: b.fun == 3
   Evaluated: 3.0 == 3,Test Passed
  Expression: b.der == 0
   Evaluated: 0.0 == 0,Test Passed
  Expression: c.fun == 10
   Evaluated: 10 == 10,Test Passed
  Expression: c.der == 1
   Evaluated: 1 == 1],false)

In [6]:
Dual(1.0)

Dual{Float64}(1.0,0.0)

---

## Ejercicio 2

Implementen *todas* las operaciones aritméticas definidas arriba. Estas operaciones deben incluir las operaciones aritméticas que involucran un número cualquiera (`a :: Real`) y un dual (`b::Dual`), como por ejemplo `a+b` o `b+a`, etc. Esto se puede hacer implementando los métodos específicos para estos casos (¡y que sirven en cualquier órden!). 

Implementen también la comparación entre duales (`==`). 

Incluyan tests que muestren que cada una de ellas está bien definida, y que sus resultados dan valores consistentes.

In [7]:
import Base: +, -, *, /, ^, ==

#= 
Aquí se implementan los métodos necesarios para cada función; 
en el caso de ^ por ahora nos conformaremos con que la potencia 
sea entera.
=#

+(A::Dual,B::Dual)=Dual(A.fun+B.fun,A.der+B.der)
+(gama::Real,B::Dual)=Dual(gama+B.fun,B.der)
+(A::Dual,gama::Real)=Dual(A.fun+gama,A.der)

-(A::Dual,B::Dual)=Dual(A.fun-B.fun,A.der-B.der)
-(delta::Real,B::Dual)=Dual(delta-B.fun,-B.der)
-(A::Dual,delta::Real)=Dual(A.fun-delta,A.der)

*(A::Dual,B::Dual)=Dual(A.fun*B.fun,(A.der*B.fun)+(A.fun*B.der))
*(alfa::Real,B::Dual)=Dual(alfa*B.fun,alfa*B.der)     #debe especificarse el orden de los factores, porque no debe alterar 
*(A::Dual,alfa::Real)=Dual(alfa*A.fun,alfa*A.der)     #el producto.

/(A::Dual,B::Dual)=Dual((A.fun)/(B.fun),((A.fun*B.der)-(A.der*B.fun))/(B.fun)^2)
/(A::Dual,epsilon::Real)=Dual(A.fun/epsilon,A.der/epsilon)

^(A::Dual,beta::Int64)=Dual((A.fun)^(beta),beta*((A.fun)^(beta-1))*(A.der))

==(A::Dual,B::Dual)=(A.fun==B.fun && A.der==B.der)
==(A::Dual,zeta::Real)=(A==zeta.fun && 0.0==zeta.der)
==(zeta::Real,B::Dual)=(zeta.fun==B.fun && zeta.der==0.0)


== (generic function with 121 methods)

In [8]:
# Aquí se incluyen las pruebas necesarias
using Base.Test
@testset "pruebas realizadas" begin
alfa=Dual(5,2)+Dual(5,2)
@test alfa.fun==10
@test alfa.der==4
alfa1=5+Dual(5,2)
@test alfa1.fun==10
@test alfa1.der==2
alfa2=Dual(5,2)+5
@test alfa2.fun==10
@test alfa2.der==2

betha=Dual(5,4)-Dual(5,2)
@test betha.fun==0
@test betha.der==2
betha1=5-Dual(5,4)
@test betha1.fun==0
@test betha1.der==-4
betha2=Dual(5,4)-5
@test betha2.fun==0
@test betha2.der==4

gamma=Dual(5,2)*Dual(6,1)
@test gamma.fun==30
@test gamma.der==17
gamma1=3*Dual(5,2)
@test gamma1.fun==15
@test gamma1.der==6
gamma2=Dual(5,2)*3
@test gamma2.fun==15
@test gamma2.der==6

delta=Dual(6,2)/Dual(2,1)
@test delta.fun==3
@test delta.der==0.5
delta1=Dual(6,2)/2
@test delta1.fun==3
@test delta1.der==1

epsilon=Dual(2)^4
@test epsilon.fun==16
@test epsilon.der==0

@test Dual(12,2)==Dual(12,2)
@test Dual(12)==Dual(12,0)
@test Dual(12,0)==Dual(12)
    
end



Test Summary:      | Pass  Total
  pruebas realizadas |   27     27


Base.Test.DefaultTestSet("pruebas realizadas",Any[Test Passed
  Expression: alfa.fun == 10
   Evaluated: 10 == 10,Test Passed
  Expression: alfa.der == 4
   Evaluated: 4 == 4,Test Passed
  Expression: alfa1.fun == 10
   Evaluated: 10 == 10,Test Passed
  Expression: alfa1.der == 2
   Evaluated: 2 == 2,Test Passed
  Expression: alfa2.fun == 10
   Evaluated: 10 == 10,Test Passed
  Expression: alfa2.der == 2
   Evaluated: 2 == 2,Test Passed
  Expression: betha.fun == 0
   Evaluated: 0 == 0,Test Passed
  Expression: betha.der == 2
   Evaluated: 2 == 2,Test Passed
  Expression: betha1.fun == 0
   Evaluated: 0 == 0,Test Passed
  Expression: betha1.der == -4
   Evaluated: -4 == -4  …  Test Passed
  Expression: gamma2.der == 6
   Evaluated: 6 == 6,Test Passed
  Expression: delta.fun == 3
   Evaluated: 3.0 == 3,Test Passed
  Expression: delta.der == 0.5
   Evaluated: 0.5 == 0.5,Test Passed
  Expression: delta1.fun == 3
   Evaluated: 3.0 == 3,Test Passed
  Expression: delta1.der == 1
   Evaluated

In [9]:
3*Dual(5,2)

Dual{Int64}(15,6)

In [10]:
xdual(1.0)

Dual{Float64}(1.0,1.0)

Sea una función como x^2

In [11]:
"""
especificando  la función a evaluar
"""
function f(x)
    return x^2
end

f

Ahora, evaluando la función con *xdual*

In [12]:
f(xdual(1.0))

Dual{Float64}(1.0,2.0)

---

## Ejercicio 3

Para reutilizar el código que han hecho en este notebook, y de hecho seguirlo desarrollando, conviene ponerlo dentro de un módulo. Para hacer esto, deberán copiar todo el código necesario (y que aparece en la resolución de los ejercicios anteriores) en un archivo cuyo nombre será "AutomDiff.jl" y cuya estructura será la siguiente

```julia
#=
Aquí viene una explicación de lo que se hace en el módulo, 
los autores y la fecha
=#

# La siguiente instrucción sirve para *precompilar* el módulo
__precompile__(true)

module AD
    import Base: +, -, *, /, ^, ==
    
    export Dual, xdual
    
    # Aquí viene TODO el código que implementaron.
    # Primero uno incluye la definición de Dual y
    # después las operaciones con Duales.
    ...

end
```

Todas las pruebas deberán ser incluidas en un archivo separado llamado "runtest.jl", y su estructura es:

```julia
# Este archivo incluye los tests del módulo AD
include("AutomDiff.jl")
using Base.test
using AD

# A continuación vienen los tests que implementaron y que deben 
# ser suficientemente exhaustivos
...

```

Estos dos archivos deben incluirlos en el envío de su tarea (además del archivo `Tarea3.ipynb`).

In [13]:
workspace()

In [14]:
include("AutomDiff.jl")

AD

In [15]:
include("runtest.jl")

Test Summary:      | Pass  Total
  pruebas realizadas |    6      6


Test Summary:      | Pass  Total
  pruebas realizadas |   27     27


Base.Test.DefaultTestSet("pruebas realizadas",Any[Test Passed
  Expression: alfa.fun == 10
   Evaluated: 10 == 10,Test Passed
  Expression: alfa.der == 4
   Evaluated: 4 == 4,Test Passed
  Expression: alfa1.fun == 10
   Evaluated: 10 == 10,Test Passed
  Expression: alfa1.der == 2
   Evaluated: 2 == 2,Test Passed
  Expression: alfa2.fun == 10
   Evaluated: 10 == 10,Test Passed
  Expression: alfa2.der == 2
   Evaluated: 2 == 2,Test Passed
  Expression: betha.fun == 0
   Evaluated: 0 == 0,Test Passed
  Expression: betha.der == 2
   Evaluated: 2 == 2,Test Passed
  Expression: betha1.fun == 0
   Evaluated: 0 == 0,Test Passed
  Expression: betha1.der == -4
   Evaluated: -4 == -4  …  Test Passed
  Expression: gamma2.der == 6
   Evaluated: 6 == 6,Test Passed
  Expression: delta.fun == 3
   Evaluated: 3.0 == 3,Test Passed
  Expression: delta.der == 0.5
   Evaluated: 0.5 == 0.5,Test Passed
  Expression: delta1.fun == 3
   Evaluated: 3.0 == 3,Test Passed
  Expression: delta1.der == 1
   Evaluated